In [18]:
# Library Imports
import yfinance as yf
import pandas as pd
import mplfinance as mpf
import matplotlib.pyplot as plt
from matplotlib import rcParams

print(mpf.__version__)

0.12.10b0


In [ ]:
# Gold (XAU/USD) データ取得
symbol = "GC=F"

df = yf.download(
    symbol,
    interval="5m",
    period="5d"
)

# MultiIndexの解除
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)

for col in ["Open", "High", "Low", "Close", "Volume"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Index名の変更
df.dropna(inplace=True)
df.index.name = "Datetime"

df['MA200'] = df['Close'].rolling(window=200).mean()

ap = [mpf.make_addplot(df['MA200'], color='red', width=1.5)]

# 時間範囲の指定
df_plot = df.last('3H')

# 範囲指定の場合
# df_plot = df['2026-01-25 09:00:00': '2026-01-25 21:00:00']

def plot_chart(df, period='3H'):
    """
    df: データフレーム（５分足
    """

    mpf.plot (
        df,
        type="candle",
        volume=True,
        style="yahoo",
        title="Gold 5-Minute",
        addplot=ap,
        width=0.6,
        datetime_format="%m/%d %H:%M:%S"
    )

[*********************100%***********************]  1 of 1 completed


AttributeError: 'DataFrame' object has no attribute 'last'